In [1]:
import comfortmodels
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML
import pandas as pd

plt.ioff()

In [2]:

DATA_URL = 'https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv'

EXPLANATION = """\
<div class="app-sidebar">
<p><em>Compare different thermal comfort conditions.</em><p>

<p>Select what thermal parameters to plot in the dropdowns, and use the sliders to set the other relevant parameters.</p>

<p>This example demonstrates combining matplotlib with Jupyter widgets. For more interactive plots,
consider using <a href="https://github.com/bloomberg/bqplot">bqplot</a>.
</div>
"""

In [3]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [4]:
VARS = ['DBT', 'MRT', 'RH', 'VEL', 'MET', 'CLO']
METRICS = ['PMV', 'PPD', 'SET']

class App:
    
    def __init__(self):
        self._x_dropdown, x_dropdown_box = self._create_indicator_dropdown(VARS, 0, label="X-axis")
        self._y_dropdown, y_dropdown_box = self._create_indicator_dropdown(VARS, 1, label="Y-axis")        
        self._metric_dropdown, metric_dropdown_box = self._create_indicator_dropdown(METRICS, 0, label="Z-axis")
        self._plot_container = widgets.Output()
        self._x_slider, x_slider_box = self._create_range_slider(
            -20, 100, "X axis range")
        self._y_slider, y_slider_box = self._create_range_slider(
            -20, 100, "Y axis range")
        self.dbt_slider, dbt_slider_box = self._create_value_slider(-20, 30, "DBT (°C)", default=24)
        self.mrt_slider, mrt_slider_box = self._create_value_slider(-20, 30, "MRT (°C)", default=24)
        self.rh_slider, rh_slider_box = self._create_value_slider(0, 100, "RH (%)", default=50)
        self.vel_slider, vel_slider_box = self._create_value_slider(0, 10, "VEL (m/s)", default=0.1)
        self.met_slider, met_slider_box = self._create_value_slider(0, 10, "MET", default=1.1)
        self.clo_slider, clo_slider_box = self._create_value_slider(0, 10, "CLO", default=0.5)
        
        self.message_label = widgets.Label("")
        
        _controls_container = widgets.HBox([
            widgets.VBox([widgets.HTML("<h3>Axis Control</h3>"), x_dropdown_box, y_dropdown_box, metric_dropdown_box, x_slider_box, y_slider_box]),
            widgets.VBox([widgets.HTML("<h3>Parameter Control</h3>"),dbt_slider_box, mrt_slider_box, rh_slider_box, vel_slider_box, met_slider_box, clo_slider_box])
        ])
        
        _app_container = widgets.VBox([
            _controls_container,
            self._plot_container,
            self.message_label,
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Thermal Comfort Plotter</h1>'
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            widgets.VBox([
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em')),
                _app_container
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        
        self._update_app()
        
    def _create_indicator_dropdown(self, indicators, initial_index, label=""):
        dropdown_label = widgets.Label(label)
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        dropdown_box = widgets.VBox([dropdown_label, dropdown])
        return dropdown, dropdown_box
    
    def _create_value_slider(self, min_val, max_val, label, default=0):
        slider_label = widgets.Label(label)
        slider = widgets.FloatSlider(
            value=default,
            min=min_val, max=max_val, step=0.1,
            layout=widgets.Layout(width='300px')
        )
        slider.observe(self._on_change, names=['value'])
        slider_box = widgets.HBox([slider_label, slider])
        return slider, slider_box
    
    def _create_range_slider(self, min_val, max_val, label, default=[10, 30]):
        slider_label = widgets.Label(label)
        slider = widgets.IntRangeSlider(
            value=default,
            min=min_val, max=max_val,
            layout=widgets.Layout(width='300px')
        )
        slider.observe(self._on_change, names=['value'])
        slider_box = widgets.HBox([slider_label, slider])
        return slider, slider_box
    
    def _create_plot(self, x_indicator, y_indicator, metric):
        
        for i, m in enumerate(METRICS):
            if metric==m:
                metricid = i
        
        C = self.plot_data(metricid)
        
        vmin=[-3,0,10] #limits for the different metrics.
        vmax=[3,100,44]
        
        levels = [[-3,-2,-1,0,1,2,3],
                  [0,10,20,30,40,50,60,70,80,90,100],
                  [10,14.5,17.5,22.2,25.6,30,34.5,37.5,44]]
        
        strs = [['Cold (-3)', 'Cool (-2)', 'Slightly Cool (-1)', 'Neutral (0)', 'Slightly Warm (1)', 'Warm (2)', 'Hot (3)'],
                ['0%','10%','20%','30%','40%','50%','60%','70%','80%','90%','100%'],
                ['Cold','Cool','Slightly Cool','Comfortable','Slightly Warm','Warm','Hot','Very Hot']]
        
        plt.xlabel(x_indicator, size=16)
        plt.ylabel(y_indicator, size=16)
        
        self._plot_container.clear_output(wait=True)
        PLOT = plt.pcolor(self.xrange, self.yrange, C,  cmap='RdBu_r', vmin=vmin[metricid], vmax=vmax[metricid])
        CS = plt.contour(self.xrange, self.yrange, C, cmap='gnuplot', levels=levels[metricid])

        fmt = {}
        for l, s in zip(CS.levels, strs[metricid]):
            fmt[l] = s

        plt.clabel(CS, inline=1, fontsize=10, fmt=fmt)
        plt.colorbar(PLOT)
        
    def build_case(self):
        case = {
              'DBT': self.dbt_slider.value,
              'MRT': self.mrt_slider.value,
              'RH': self.rh_slider.value,
              'VEL': self.vel_slider.value,
              'MET': self.met_slider.value,
              'CLO': self.clo_slider.value,
              }
        return case
        
    def _on_change(self, _):
        self._update_app()
        
    def plot_data(self, metricid):
        case = self.build_case()
        xrange = np.linspace(self._x_slider.value[0], self._x_slider.value[1], 20)
        yrange = np.linspace(self._y_slider.value[0], self._y_slider.value[1], 20)
        self.xrange = xrange
        self.yrange = yrange
        C=np.zeros((len(xrange),len(yrange)))

        xaxis = self._x_dropdown.value
        yaxis = self._y_dropdown.value

        for i, x in enumerate(xrange):
            case[xaxis] = xrange[i]
            for j, y in enumerate(yrange):
                case[yaxis] = yrange[j]
                if metricid == 0:
                    C[j,i] = comfortmodels.pmvElevatedAirspeed(case['DBT'], case['MRT'], case['VEL'], case['RH'], case['MET'], case['CLO'], 0.0)['pmv']
                elif metricid == 1:
                    C[j,i] = comfortmodels.pmvElevatedAirspeed(case['DBT'], case['MRT'], case['VEL'], case['RH'], case['MET'], case['CLO'], 0.0)['ppd']
                elif metricid == 2:
                    C[j,i] = comfortmodels.pierceSET(case['DBT'], case['MRT'], case['VEL'], case['RH'], case['MET'], case['CLO'], 0.0)
                
        return C
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        metric = self._metric_dropdown.value
        
        self.message_label.value = "Updating..."
        with self._plot_container:
            self._create_plot(x_indicator, y_indicator, metric)
            plt.show()
        self.message_label.value = ""

In [24]:

app = App()

app.container

In [5]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 
btn_rm = wg.Button(description = 'Remove') 
btn_print = wg.Button(description = "Count Children")

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

def on_rm_bttn_clicked(b):
    print("h")
    vb.children = tuple(list(vb.children)[:-1])
    
def on_p_bttn_clicked(b):
    b.description = "Count Children ({0})".format(len(list(vb.children)))
'''    
btn.on_click(on_bttn_clicked)
btn_rm.on_click(on_rm_bttn_clicked)
btn_print.on_click(on_p_bttn_clicked)
display(btn, btn_rm, btn_print, vb)

list(vb.children)
'''

'    \nbtn.on_click(on_bttn_clicked)\nbtn_rm.on_click(on_rm_bttn_clicked)\nbtn_print.on_click(on_p_bttn_clicked)\ndisplay(btn, btn_rm, btn_print, vb)\n\nlist(vb.children)\n'